In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model


In [12]:
word_index = imdb.get_word_index()
# word_index
reverse_word_index = {value : key for key, value in word_index.items()}

In [13]:
model = load_model("Simple_RNN_.h5")

In [14]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [15]:
model.get_weights()

[array([[ 0.10137738,  0.20136409, -0.03071438, ...,  0.12040153,
          0.10328982,  0.09628826],
        [-0.00156642,  0.03022273,  0.01950423, ...,  0.01897645,
         -0.03824544,  0.02395835],
        [-0.04562633,  0.0129656 , -0.15472677, ...,  0.1010664 ,
          0.07537075,  0.03853465],
        ...,
        [-0.01741997, -0.02793813,  0.00981286, ..., -0.00098575,
          0.01147409,  0.00845417],
        [ 0.03084252,  0.06804293,  0.079931  , ...,  0.02351114,
          0.02824788, -0.02776877],
        [ 0.2621472 ,  0.24511765,  0.07107521, ...,  0.07893575,
          0.0350591 ,  0.12567645]], shape=(10000, 128), dtype=float32),
 array([[ 0.05545266,  0.08590235, -0.11341365, ...,  0.15895662,
         -0.08894027,  0.11139058],
        [ 0.00781918,  0.17814784,  0.01590594, ..., -0.12504564,
         -0.0990068 ,  0.05245434],
        [ 0.00703807, -0.14641784, -0.10618901, ...,  0.11296759,
         -0.18636957,  0.05546444],
        ...,
        [-0.0768574

In [16]:
# Helper functions
# Function to decode Review
def decode_review(encoded_review):
    return " ".join([reverse_word_index.get(i-3,'?') for i in encoded_review])

# Function preprocessing user input 
def perprocessing_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word,2)+3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [17]:
# prediction functions
def predict_sentiments(review):
    preprocessed_input = perprocessing_text(review)
    prediction = model.predict(preprocessed_input)
    sentiments = "Positive" if prediction[0][0] > 0.05 else "Negative"
    return sentiments, prediction[0][0]

In [18]:
example_review = "This movie was fantastic! The acting was great and the plot was thrilling"

sentiment, score = predict_sentiments(example_review)
print(f"Review: {example_review}")
print(f"Sentiment: {sentiment}")
print(f"Prediciton Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling
Sentiment: Positive
Prediciton Score: 0.9999440908432007


In [1]:
# a = "this movie is bullshit and i just hate this movie"
# sentiment, score = predict_sentiments(a)
# print(f"Review: {a}")
# print(f"Sentiment: {sentiment}")
# print(f"Prediciton Score: {score}")

In [3]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# --- Config ---
max_features = 10000      # vocab size you trained with
maxlen = 200              # sequence length used during training

# Load tokenizer-compatible IMDB data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Load trained model
model = load_model("Simple_RNN_.h5")
# --- Prediction helper ---
def predict_sentiments(review, word_index=imdb.get_word_index()):
    # Convert text -> word indices
    words = review.lower().split()
    seq = []
    for w in words:
        idx = word_index.get(w, 2)  # 2 is <OOV> in IMDB
        # clamp index if it exceeds max_features
        if idx >= max_features:
            idx = 2  # map to <OOV>
        seq.append(idx)

    # Pad to training length
    seq = pad_sequences([seq], maxlen=maxlen)

    # Predict
    score = model.predict(seq, verbose=0)[0][0]
    sentiment = "Positive" if score >= 0.5 else "Negative"

    return sentiment, float(score)


In [4]:
a = "this movie is bullshit and i just hate this movie"
sentiment, score = predict_sentiments(a)
print(f"Review: {a}")
print(f"Sentiment: {sentiment}")
print(f"Prediction Score: {score:.4f}")


Review: this movie is bullshit and i just hate this movie
Sentiment: Positive
Prediction Score: 0.9999
